In [9]:
from __future__ import division,print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

import utils; reload(utils)
from utils import plots

import vgg16; reload(vgg16)
from vgg16 import Vgg16

%matplotlib inline

In [10]:
#preprocessing label files to accumulate the data(bmp,jpg,png,jpeg)

import os
import pandas as pd
rootdir = '/home/vijin/DeepLearning/PetaData/PETA dataset'

male_file_map= {}
female_file_map = {}

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if 'Label.txt' in file:
            male_file_paths = []
            female_file_paths = []
            with open(os.path.join(subdir, file)) as label_file:
                for line in label_file:
                    if 'personalMale' in line:
                        male_file_paths.append(line.split(' ')[0])
                    else:
                        female_file_paths.append(line.split(' ')[0])
            male_file_map[os.path.join(subdir)] = set(male_file_paths)
            female_file_map[os.path.join(subdir)] = set(female_file_paths)
            

In [11]:
import re
ml_files = []
for key in male_file_map.keys():
    for subdir, dirs, files in os.walk(key):
        for file in files:
            if re.match(r'.*(jpg|bmp|jpeg|png)$', str(file)):
                for id in male_file_map[key]:
                    #CUHK Dataset
                    if 'png' in id:
                        if str(file) == id :
                            ml_files.append(os.path.join(subdir, file))
                    else:
                        if str(file).split('_')[0] == id :
                            ml_files.append(os.path.join(subdir, file)) 

In [13]:
fl_files = []
for key in female_file_map.keys():
    for subdir, dirs, files in os.walk(key):
        for file in files:
            if re.match(r'.*(jpg|bmp|jpeg|png)$', str(file)):
                for id in female_file_map[key]:
                    #CUHK Dataset
                    if 'png' in id:
                        if str(file) == id :
                            fl_files.append(os.path.join(subdir, file))
                    else:
                        if str(file).split('_')[0] == id :
                            fl_files.append(os.path.join(subdir, file)) 

In [14]:
#shuffle and split
import random

random.seed(7)
random.shuffle(ml_files)
random.shuffle(fl_files)

valid_ml_files = ml_files[0:int(len(ml_files)*.10)]
train_ml_files = ml_files[int(len(ml_files)*.10):]

valid_fl_files = fl_files[0:int(len(fl_files)*.10)]
train_fl_files = fl_files[int(len(fl_files)*.10):]

In [16]:
#copy files
import shutil

for train_ml in train_ml_files:
    shutil.copy(train_ml, '/home/vijin/DeepLearning/PetaData/data/train/male')
    
for valid_ml in valid_fl_files:
    shutil.copy(valid_ml, '/home/vijin/DeepLearning/PetaData/data/valid/male')
    
    
for train_fl in train_fl_files:
    shutil.copy(train_fl, '/home/vijin/DeepLearning/PetaData/data/train/female')
    
for valid_fl in valid_fl_files:
    shutil.copy(valid_fl, '/home/vijin/DeepLearning/PetaData/data/valid/female')

In [19]:
#cnn with vgg16
path = "/home/vijin/DeepLearning/PetaData/data/"
batch_size = 64

vgg = Vgg16()
batches = vgg.get_batches(path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size*2)
vgg.finetune(batches)

vgg.model.optimizer.lr = 0.01

vgg.fit(batches, val_batches, nb_epoch=1)
vgg.model.save_weights(path+'result/ft1.h5')

/home/vijin/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:577: UserWarning: `output_shape` argument not specified for layer lambda_2 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 224, 224)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


Found 16957 images belonging to 2 classes.
Found 1712 images belonging to 2 classes.
Epoch 1/1
16957/16957 [==============================] - 295s - loss: 5.6782 - acc: 0.6095 - val_loss: 7.0812 - val_acc: 0.5000

In [20]:
vgg.fit(batches, val_batches, nb_epoch=1)
vgg.model.save_weights(path+'result/ft2.h5')

Epoch 1/1
16957/16957 [==============================] - 295s - loss: 5.0313 - acc: 0.6463 - val_loss: 7.3837 - val_acc: 0.5023

In [21]:
vgg.fit(batches, val_batches, nb_epoch=1)
vgg.model.save_weights(path+'result/ft3.h5')

Epoch 1/1
16957/16957 [==============================] - 293s - loss: 4.9487 - acc: 0.6548 - val_loss: 7.3677 - val_acc: 0.4971

In [22]:
vgg.fit(batches, val_batches, nb_epoch=1)
vgg.model.save_weights(path+'result/ft4.h5')

Epoch 1/1
16957/16957 [==============================] - 292s - loss: 4.7431 - acc: 0.6701 - val_loss: 7.0841 - val_acc: 0.5006

In [23]:
vgg.fit(batches, val_batches, nb_epoch=1)
vgg.model.save_weights(path+'result/ft5.h5')

Epoch 1/1
16957/16957 [==============================] - 293s - loss: 4.8797 - acc: 0.6678 - val_loss: 7.4680 - val_acc: 0.5076

In [24]:
vgg.fit(batches, val_batches, nb_epoch=5)
vgg.model.save_weights(path+'result/ft6_5.h5')

Epoch 1/5
16957/16957 [==============================] - 292s - loss: 4.9054 - acc: 0.6677 - val_loss: 7.4793 - val_acc: 0.5029

In [25]:
vgg.fit(batches, val_batches, nb_epoch=20)
vgg.model.save_weights(path+'result/ft7_20.h5')

Epoch 1/20
16957/16957 [==============================] - 292s - loss: 4.8527 - acc: 0.6787 - val_loss: 7.6698 - val_acc: 0.4953